In [17]:
import pandas as pd
import re
import datetime as dt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
# from plotnine import *
# to avoid warnings
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

In [32]:
item_dataset = pd.read_csv('data/item_data.csv',index_col=0)
top_items = item_dataset
item_dataset.head()

,item_postcode,item_lat,item_lng,item_category,item_name,item_condition
item_id,,,,,,
1,12687,52.499900,13.389746,appliance,kettle,medium
2,10717,52.466777,13.368009,furniture,mirror,good
3,10318,52.539631,13.329739,clothes,woman_clothes,medium
4,10713,52.473807,13.361731,furniture,mirror,good
5,12277,52.487921,13.582011,kids,stroller,medium


In [33]:
item_dataset = data

### Exploring distances for user looker and item available

In [34]:
# Fake user data
user_coordinates=[52.52646152725616, 13.376623742933933]
lat1 = np.radians(52.52646152725616)
lon1 = np.radians(13.376623742933933)

In [35]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0
data['lat2'] = np.radians(data['item_lat'])
data['lng2'] = np.radians(data['item_lng'])

data['dng'] = data['lng2'] - lon1
data['dlat'] = data['lat2'] - lat1

data['a'] = np.sin(data['dlat'] / 2)**2 + cos(lat1) * np.cos(data['lat2']) * np.sin(data['dlon'] / 2)**2
data['c'] = 2 * np.arctan2(np.sqrt(data['a'] ), np.sqrt(1 - data['a'] ))

data['geo_distance'] = R * data['c']

data['geo_distance']

item_id
365     10.721495
683      8.209928
448     13.602505
1125     7.670831
1172     6.620127
          ...    
499     10.869366
877      2.956106
1375    14.456377
922      4.799018
1218     5.369159
Name: geo_distance, Length: 1500, dtype: float64

In [36]:
data['geo_distance'].nsmallest(5)

item_id
796     0.069879
438     0.325348
1258    0.452808
515     0.469925
173     0.537221
Name: geo_distance, dtype: float64

### Exploring clustering with kmeans

#### Using silhoette score to estimate the best k and number of clusters

In [38]:
coords = top_items[['item_lat','item_lng']]

distortions = []
sil = []
K = range(2,100)
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    distortions.append(kmeansModel.inertia_)
    labels = kmeansModel.labels_
    sil.append(silhouette_score(coords, labels,metric = 'euclidean'))

sse_df =pd.DataFrame({"K":K,
                      'sse':distortions,
                      'silhouette':sil})
(ggplot(sse_df, aes(x='K', y='silhouette'))+geom_point()+geom_line()+theme_minimal()+labs(title='silhouette score for different K values'))

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values